Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback): # Your Code):
  # Your Code
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 99.9% training accuracy so cancelling training")
      self.model.stop_training = True

callbacks = myCallback()

--2019-07-09 07:19:15--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 2a00:1450:400c:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-07-09 07:19:20 (138 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
!ls -a /tmp/h-or-s/happy/

.	       happy1-07.png  happy1-16.png  happy2-05.png  happy2-14.png
..	       happy1-08.png  happy1-17.png  happy2-06.png  happy2-15.png
happy1-00.png  happy1-09.png  happy1-18.png  happy2-07.png  happy2-16.png
happy1-01.png  happy1-10.png  happy1-19.png  happy2-08.png  happy2-17.png
happy1-02.png  happy1-11.png  happy2-00.png  happy2-09.png  happy2-18.png
happy1-03.png  happy1-12.png  happy2-01.png  happy2-10.png  happy2-19.png
happy1-04.png  happy1-13.png  happy2-02.png  happy2-11.png
happy1-05.png  happy1-14.png  happy2-03.png  happy2-12.png
happy1-06.png  happy1-15.png  happy2-04.png  happy2-13.png


In [3]:
# Your Code
"""
Get information about images in a folder.
"""
from os import listdir
from os.path import isfile, join

from PIL import Image


def print_data(data):
  """
    data : dict
  """
  for k, v in data.items():
    print("%s:\t%s" % (k, v))
  print("Min width: %i" % data['min_width'])
  print("Max width: %i" % data['max_width'])
  print("Min height: %i" % data['min_height'])
  print("Max height: %i" % data['max_height'])


def print_images_size(path):
  """
  path : str
      Path where to look for image files.
  """
  onlyfiles = [join(path,f) for f in listdir(path) if isfile(join(path, f))]
  
  onlyfiles = [f for f in onlyfiles if f.endswith('.png')]

  data = {}
  data['images_count'] = len(onlyfiles)
  data['min_width'] = 10**100  # No image will be bigger than that
  data['max_width'] = 0
  data['min_height'] = 10**100  # No image will be bigger than that
  data['max_height'] = 0

  for filename in onlyfiles:
    im = Image.open(filename)
    width, height = im.size
    data['min_width'] = min(width, data['min_width'])
    data['max_width'] = max(width, data['max_height'])
    data['min_height'] = min(height, data['min_height'])
    data['max_height'] = max(height, data['max_height'])
    
  print_data(data)
  
print_images_size("/tmp/h-or-s/happy/")
print_images_size("/tmp/h-or-s/sad/")

# Your Code

images_count:	40
min_width:	150
max_width:	150
min_height:	150
max_height:	150
Min width: 150
Max width: 150
Min height: 150
Max height: 150
images_count:	40
min_width:	150
max_width:	150
min_height:	150
max_height:	150
Min width: 150
Max width: 150
Min height: 150
Max height: 150


In [4]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
# Your Code Here
    tf.keras.layers.Conv2D(16, (3,3), activation = tf.nn.relu, input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),loss='binary_crossentropy', metrics=['acc'])# Your Code Here #)

W0709 07:19:33.303796 140012727052160 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0709 07:19:33.545802 140012727052160 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [6]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)# Your Code Here

train_generator = train_datagen.flow_from_directory(
        # Your Code Here)
    "/tmp/h-or-s/",
    target_size=(150,150),
    batch_size=16,
    class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [7]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
      # Your Code Here)
    train_generator,
    steps_per_epoch = 5,
    epochs= 15,
    callbacks=[callbacks],
    verbose=1)
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
5/5 [==============================] - 4s 761ms/step - loss: 3.5712 - acc: 0.4000
Epoch 2/15
5/5 [==============================] - 0s 42ms/step - loss: 0.7733 - acc: 0.4625
Epoch 3/15
5/5 [==============================] - 0s 39ms/step - loss: 0.6282 - acc: 0.6250
Epoch 4/15
5/5 [==============================] - 0s 37ms/step - loss: 0.6083 - acc: 0.6375
Epoch 5/15
5/5 [==============================] - 0s 40ms/step - loss: 0.5203 - acc: 0.7750
Epoch 6/15
5/5 [==============================] - 0s 41ms/step - loss: 0.4322 - acc: 0.8000
Epoch 7/15
5/5 [==============================] - 0s 37ms/step - loss: 0.3275 - acc: 0.9375
Epoch 8/15
3/5 [=================>............] - ETA: 0s - loss: 0.1180 - acc: 1.0000
Reached 99.9% training accuracy so cancelling training
5/5 [==============================] - 0s 39ms/step - loss: 0.0825 - acc: 1.0000
